Weights and Biases using keras

Install libraries 

In [1]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 36.4 MB/s 
     |████████████████████████████████| 144 kB 41.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 


Login to account and get the API

In [2]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

1. Start a new run and pass hyperparameters to track
2. log metrics from training
3. visualize reults in dashboard


Run the experiemnt 
1. launch 5 similated experiments
2. set entity, specify username
3. set project name
4. track hyperparamerts 
5. calculate accuracy

In [3]:
import random

#5 simulated experiments
for run in range(5):
  wandb.init(
      project="basic-intro", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",})
  
  offset = random.random() / 5
  for ii in range(2, 10):
      acc = 1 - 2 ** -ii - random.random() / ii - offset
      loss = 2 ** -ii + random.random() / ii + offset
      wandb.log({"acc": acc, "loss": loss})
      
  # run finished
  wandb.finish()

wandb: Currently logged in as: harikanalam (use `wandb login --relogin` to force relogin)


acc,▁▃▇▇▇▇██
loss,██▄▅▂▃▃▁
acc,0.87504
loss,0.10422


acc,▁▂▆▄▅█▆▆
loss,█▅▄▂▂▁▁▁
acc,0.82208
loss,0.1716


acc,▁▃▅▅████
loss,█▆▃▅▁▂▂▁
acc,0.79986
loss,0.191


acc,▁▁▇▄████
loss,█▅▆▅▂▁▁▂
acc,0.82563
loss,0.18434


acc,▁▅▃▅▅▅██
loss,█▅▂▃▁▁▂▂
acc,0.96182
loss,0.12011


In [4]:
%wandb stacey/deep-drive/runs/1wyssjcx -h 720

Simple Keras DNN.  
Run model to train a simple MNIST classifier.    
Launch 20 experiments with different dropout rates. track huperparameters. set name ans project name.

Load MNISt data and train model and test the model.

In [5]:
import random

import numpy as np
import tensorflow as tf
from wandb.keras import WandbCallback


for run in range(20):
  wandb.init(
      project="keras-intro",
      config={
          "layer_1": 512,
          "activation_1": "relu",
          "dropout": random.uniform(0.01, 0.80),
          "layer_2": 10,
          "activation_2": "softmax",
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 6,
          "batch_size": 256
      })
  config = wandb.config

  # Get the data
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  x_train, y_train = x_train[::5], y_train[::5]  # Subset data for a faster demo
  x_test, y_test = x_test[::20], y_test[::20]
  labels = [str(digit) for digit in range(np.max(y_train) + 1)]

  # Build a model
  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
      tf.keras.layers.Dropout(config.dropout),
      tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
      ])

  model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

  # WandbCallback auto-saves all metrics from model.fit(), plus predictions on validation_data
  logging_callback = WandbCallback(log_evaluation=True)

  history = model.fit(x=x_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[logging_callback]
                      )

  # Mark the run as finished
  wandb.finish()

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/6
47/47 [==============================] - 2s 23ms/step - loss: 2.2554 - accuracy: 0.1912 - val_loss: 1.8550 - val_accuracy: 0.6000 - _timestamp: 1648708012.0000 - _runtime: 25.0000
Epoch 2/6
47/47 [==============================] - 1s 18ms/step - loss: 1.8688 - accuracy: 0.3704 - val_loss: 1.5521 - val_accuracy: 0.7100 - _timestamp: 1648708013.0000 - _runtime: 26.0000
Epoch 3/6
47/47 [==============================] - 1s 17ms/step - loss: 1.6090 - accuracy: 0.4980 - val_loss: 1.3268 - val_accuracy: 0.7500 - _timestamp: 1648708013.0000 - _runtime: 26.0000
Epoch 4/6
47/47 [==============================] - 1s 18ms/step - loss: 1.4319 - accuracy: 0.5687 - val_loss: 1.1614 - val_accuracy: 0.7680 - _timestamp: 1648708014.0000 - _runtime: 27.0000
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.2832 - accuracy: 0.6208 - val_loss: 1.0338 - val_accuracy: 0.7840 - _timestamp: 1648708015.0000 - _ru

accuracy,▁▄▆▇██
epoch,▁▂▄▅▇█
loss,█▅▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.651
best_epoch,5
best_val_loss,0.93565
epoch,5
loss,1.18876
val_accuracy,0.798


Epoch 1/6
47/47 [==============================] - 1s 22ms/step - loss: 2.1804 - accuracy: 0.2194 - val_loss: 1.8711 - val_accuracy: 0.5940 - _timestamp: 1648708034.0000 - _runtime: 8.0000
Epoch 2/6
47/47 [==============================] - 1s 19ms/step - loss: 1.7773 - accuracy: 0.4652 - val_loss: 1.5309 - val_accuracy: 0.7000 - _timestamp: 1648708035.0000 - _runtime: 9.0000
Epoch 3/6
47/47 [==============================] - 1s 19ms/step - loss: 1.4983 - accuracy: 0.5920 - val_loss: 1.2806 - val_accuracy: 0.7740 - _timestamp: 1648708036.0000 - _runtime: 10.0000
Epoch 4/6
47/47 [==============================] - 1s 18ms/step - loss: 1.2909 - accuracy: 0.6616 - val_loss: 1.0981 - val_accuracy: 0.7900 - _timestamp: 1648708037.0000 - _runtime: 11.0000
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.1452 - accuracy: 0.6992 - val_loss: 0.9656 - val_accuracy: 0.8100 - _timestamp: 1648708038.0000 - _runtime: 12.0000
Epoch 6/6
47/47 [==============================] - 1

accuracy,▁▄▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▄▇▇██
val_loss,█▆▄▃▂▁
accuracy,0.72533
best_epoch,5
best_val_loss,0.8673
epoch,5
loss,1.02066
val_accuracy,0.818


Epoch 1/6
47/47 [==============================] - 1s 22ms/step - loss: 2.1508 - accuracy: 0.2513 - val_loss: 1.8419 - val_accuracy: 0.6380 - _timestamp: 1648708057.0000 - _runtime: 9.0000
Epoch 2/6
47/47 [==============================] - 1s 19ms/step - loss: 1.7238 - accuracy: 0.5353 - val_loss: 1.4934 - val_accuracy: 0.7320 - _timestamp: 1648708058.0000 - _runtime: 10.0000
Epoch 3/6
47/47 [==============================] - 1s 19ms/step - loss: 1.4274 - accuracy: 0.6587 - val_loss: 1.2395 - val_accuracy: 0.7820 - _timestamp: 1648708058.0000 - _runtime: 10.0000
Epoch 4/6
47/47 [==============================] - 1s 19ms/step - loss: 1.2203 - accuracy: 0.7123 - val_loss: 1.0600 - val_accuracy: 0.8060 - _timestamp: 1648708059.0000 - _runtime: 11.0000
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.0705 - accuracy: 0.7445 - val_loss: 0.9293 - val_accuracy: 0.8240 - _timestamp: 1648708060.0000 - _runtime: 12.0000
Epoch 6/6
47/47 [==============================] - 

...

accuracy,▁▅▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▇███
val_loss,█▆▄▃▂▁
accuracy,0.77067
best_epoch,5
best_val_loss,0.84034
epoch,5
loss,0.95384
val_accuracy,0.808


Epoch 1/6
47/47 [==============================] - 1s 21ms/step - loss: 2.3030 - accuracy: 0.1645 - val_loss: 1.9212 - val_accuracy: 0.5840 - _timestamp: 1648708420.0000 - _runtime: 12.0000
Epoch 2/6
47/47 [==============================] - 1s 18ms/step - loss: 1.9104 - accuracy: 0.3537 - val_loss: 1.6064 - val_accuracy: 0.7160 - _timestamp: 1648708421.0000 - _runtime: 13.0000
Epoch 3/6
47/47 [==============================] - 1s 18ms/step - loss: 1.6357 - accuracy: 0.4996 - val_loss: 1.3660 - val_accuracy: 0.7640 - _timestamp: 1648708422.0000 - _runtime: 14.0000
Epoch 4/6
47/47 [==============================] - 1s 18ms/step - loss: 1.4392 - accuracy: 0.5818 - val_loss: 1.1847 - val_accuracy: 0.7880 - _timestamp: 1648708423.0000 - _runtime: 15.0000
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.2873 - accuracy: 0.6263 - val_loss: 1.0471 - val_accuracy: 0.8080 - _timestamp: 1648708424.0000 - _runtime: 16.0000
Epoch 6/6
47/47 [==============================] -

accuracy,▁▄▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.65617
best_epoch,5
best_val_loss,0.94288
epoch,5
loss,1.18218
val_accuracy,0.818


Epoch 1/6
47/47 [==============================] - 2s 34ms/step - loss: 2.1221 - accuracy: 0.2940 - val_loss: 1.8632 - val_accuracy: 0.5660 - _timestamp: 1648708442.0000 - _runtime: 12.0000
Epoch 2/6
47/47 [==============================] - 1s 19ms/step - loss: 1.6970 - accuracy: 0.6042 - val_loss: 1.5017 - val_accuracy: 0.6940 - _timestamp: 1648708443.0000 - _runtime: 13.0000
Epoch 3/6
47/47 [==============================] - 1s 18ms/step - loss: 1.3909 - accuracy: 0.7063 - val_loss: 1.2416 - val_accuracy: 0.7500 - _timestamp: 1648708444.0000 - _runtime: 14.0000
Epoch 4/6
47/47 [==============================] - 1s 19ms/step - loss: 1.1736 - accuracy: 0.7542 - val_loss: 1.0592 - val_accuracy: 0.7780 - _timestamp: 1648708445.0000 - _runtime: 15.0000
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.0176 - accuracy: 0.7871 - val_loss: 0.9290 - val_accuracy: 0.7940 - _timestamp: 1648708446.0000 - _runtime: 16.0000
Epoch 6/6
47/47 [==============================] -

accuracy,▁▅▇▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇▇█
val_loss,█▆▄▃▂▁
accuracy,0.80033
best_epoch,5
best_val_loss,0.83486
epoch,5
loss,0.90584
val_accuracy,0.812
